In [23]:
from ase.io import read
from ase.visualize import view
crystals = ['TiN', 'VN', 'ScN', 'ZrN', 'NbN']  # Assuming these are defined elsewhere

for crystal_name in crystals:
    filename = f'{crystal_name}_PBE+D3_strain.traj'
    crystal = read(filename,index=':')  # Read the last frame of the trajectory file
    #print(crystal.cell.cellpar())
    view(crystal)  # This will open a window to visualize the crystal structure
    print(f"Visualizing {crystal_name} structure from {filename}")

Visualizing TiN structure from TiN_PBE+D3_strain.traj
Visualizing VN structure from VN_PBE+D3_strain.traj
Visualizing ScN structure from ScN_PBE+D3_strain.traj
Visualizing ZrN structure from ZrN_PBE+D3_strain.traj
Visualizing NbN structure from NbN_PBE+D3_strain.traj


In [2]:
from ase.io import read, write
from ase.atoms import Atoms
import os

materials = ['TiN', 'VN', 'ScN', 'NbN', 'ZrN']

for material in materials:
    print(f"Processing {material} to fix atom indexing...")
    

    crystal = read(material + '_PBE+D3_strain.traj')  # Read the crystal structure from CIF file

    # Get a list of atoms with their chemical symbols and original indices
    # We will sort this list to create a new Atoms object with desired order
    atom_list = []
    for atom in crystal:
        atom_list.append((atom.symbol, atom.position, atom.tag, atom.momentum, atom.mass, atom.magmom))
    
    # Sort atoms first by chemical symbol (e.g., 'N' before 'Nb', 'Ti' before 'N')
    # This assumes consistent naming for metal and nitrogen atoms.
    # For example, if 'M' is the metal and 'N' is nitrogen, 'M' will come before 'N' due to alphabetical sort.
    atom_list.sort(key=lambda x: x[0]) 

    # Create a new Atoms object with the reordered atoms
    # Extract positions, symbols, and cell/pbc from original crystal
    new_symbols = [item[0] for item in atom_list]
    new_positions = [item[1] for item in atom_list]
    
    reordered_crystal = Atoms(symbols=new_symbols,
                              positions=new_positions,
                              cell=crystal.get_cell(),
                              pbc=crystal.get_pbc())

    # Add tags, momentum, mass, magmom if they were present
    if crystal.has('tags'):
        reordered_crystal.set_tags([item[2] for item in atom_list])
    if crystal.has('momenta'):
        reordered_crystal.set_momenta([item[3] for item in atom_list])
    if crystal.has('masses'):
        reordered_crystal.set_masses([item[4] for item in atom_list])
    if crystal.has('magmoms'):
        reordered_crystal.set_magmoms([item[5] for item in atom_list])

    # Save the reordered crystal to an .xyz file
    xyz_filename =  f'{material}.xyz'
    write(xyz_filename, reordered_crystal)
    
    print(f"Reordered atoms and saved to {xyz_filename}")
    print(f"New atom order for {material}: {reordered_crystal.get_chemical_formula()} (original number of atoms: {len(crystal)})")

print("\nAtom indexing fix process completed for all materials.")

Processing TiN to fix atom indexing...
Reordered atoms and saved to TiN.xyz
New atom order for TiN: N4Ti4 (original number of atoms: 8)
Processing VN to fix atom indexing...
Reordered atoms and saved to VN.xyz
New atom order for VN: N4V4 (original number of atoms: 8)
Processing ScN to fix atom indexing...
Reordered atoms and saved to ScN.xyz
New atom order for ScN: N4Sc4 (original number of atoms: 8)
Processing NbN to fix atom indexing...
Reordered atoms and saved to NbN.xyz
New atom order for NbN: N4Nb4 (original number of atoms: 8)
Processing ZrN to fix atom indexing...
Reordered atoms and saved to ZrN.xyz
New atom order for ZrN: N4Zr4 (original number of atoms: 8)

Atom indexing fix process completed for all materials.


In [6]:
from ase.io import read, write
from ase.atoms import Atoms
import os

materials = ['TiN', 'VN', 'ScN', 'NbN', 'ZrN']

for material in materials:
    print(f"Processing {material} to fix atom indexing...")
    
    # Define the input .xyz filename
    xyz_input_filename = material + '.xyz'

    try:
        # Read the crystal structure from the .xyz file
        # .xyz files typically contain a single structure, so no index=':' is needed.
        crystal = read(xyz_input_filename) 
    except FileNotFoundError:
        print(f"Error: File '{xyz_input_filename}' not found. Skipping.")
        continue # Skip to the next material

    # Check which atom properties are present in the original crystal
    has_tags = crystal.has('tags')
    has_momenta = crystal.has('momenta')
    has_masses = crystal.has('masses')
    has_magmoms = crystal.has('magmoms')

    # Get a list of tuples containing atom symbol, position, and other properties
    atom_list = []
    for atom in crystal:
        atom_list.append((
            atom.symbol,
            atom.position,
            atom.tag if has_tags else None,       # Store tag or None if not present
            atom.momentum if has_momenta else None, # Store momentum or None if not present
            atom.mass if has_masses else None,     # Store mass or None if not present
            atom.magmom if has_magmoms else None   # Store magmom or None if not present
        ))
    
    # Sort atoms alphabetically based on their chemical symbol
    atom_list.sort(key=lambda x: x[0]) 

    # Create a new Atoms object with the reordered symbols and positions
    new_symbols = [item[0] for item in atom_list]
    new_positions = [item[1] for item in atom_list]
    
    reordered_crystal = Atoms(symbols=new_symbols,
                              positions=new_positions,
                              cell=crystal.get_cell(), # Preserve original cell information
                              pbc=crystal.get_pbc())    # Preserve original periodic boundary conditions

    # Reassign other properties (tags, momenta, masses, magmoms) to the reordered crystal
    # Ensure properties are only set if they were present in the original file
    if has_tags:
        reordered_crystal.set_tags([item[2] for item in atom_list])
    if has_momenta:
        reordered_crystal.set_momenta([item[3] for item in atom_list])
    if has_masses:
        reordered_crystal.set_masses([item[4] for item in atom_list])
    if has_magmoms:
        reordered_crystal.set_magmoms([item[5] for item in atom_list])

    # Define the output .xyz filename
    xyz_output_filename = f'{material}_reordered.xyz'
    
    # Save the reordered crystal to the new .xyz file
    write(xyz_output_filename, reordered_crystal)
    
    print(f"Reordered atoms and saved to {xyz_output_filename}")
    print(f"New atom order for {material}: {reordered_crystal.get_chemical_formula()} (original number of atoms: {len(crystal)})")

print("\nAtom indexing fix process completed for all materials.")

Processing TiN to fix atom indexing...
Reordered atoms and saved to TiN_reordered.xyz
New atom order for TiN: N4Ti4 (original number of atoms: 8)
Processing VN to fix atom indexing...
Reordered atoms and saved to VN_reordered.xyz
New atom order for VN: N4V4 (original number of atoms: 8)
Processing ScN to fix atom indexing...
Reordered atoms and saved to ScN_reordered.xyz
New atom order for ScN: N4Sc4 (original number of atoms: 8)
Processing NbN to fix atom indexing...
Reordered atoms and saved to NbN_reordered.xyz
New atom order for NbN: N4Nb4 (original number of atoms: 8)
Processing ZrN to fix atom indexing...
Reordered atoms and saved to ZrN_reordered.xyz
New atom order for ZrN: N4Zr4 (original number of atoms: 8)

Atom indexing fix process completed for all materials.


In [13]:
from ase.io import read, write
import numpy as np
from ase.geometry import get_fractional_coordinates, get_scaled_positions
from ase import Atoms

# Reference nitrogen positions (fractional coordinates)
# Adjust or extend this based on your structure
reference_frac_positions = np.array([
    [0, 0, 1],  # (001)
    [1, 0, 0],  # (100)
    [0, 1, 0],  # (010)
    [1, 1, 1],  # (111)
])

def reorder_nitrogens_by_reference(input_xyz, output_xyz, metal_symbol='Zr'):
    atoms = read(input_xyz)
    cell = atoms.get_cell()
    pbc = atoms.get_pbc()

    # Separate atoms
    metal_atoms = [atom for atom in atoms if atom.symbol == metal_symbol]
    nitrogen_atoms = [atom for atom in atoms if atom.symbol == 'N']
    nitrogen_positions_frac = atoms.get_scaled_positions()[[i for i, a in enumerate(atoms) if a.symbol == 'N']]

    reordered_nitrogens = []

    for ref in reference_frac_positions:
        # Find the nitrogen closest to the reference fractional position
        distances = np.linalg.norm(np.mod(nitrogen_positions_frac - ref + 1.5, 1.0) - 0.5, axis=1)
        idx = np.argmin(distances)
        reordered_nitrogens.append(nitrogen_atoms[idx])
        # Remove matched nitrogen to avoid duplicates
        nitrogen_atoms.pop(idx)
        nitrogen_positions_frac = np.delete(nitrogen_positions_frac, idx, axis=0)

    # Append any remaining nitrogens (if more than 4 in structure)
    reordered_nitrogens += nitrogen_atoms

    reordered_atoms = Atoms(reordered_nitrogens + metal_atoms)
    reordered_atoms.set_cell(cell)
    reordered_atoms.set_pbc(pbc)

    write(output_xyz, reordered_atoms)
    print(f"{output_xyz} saved with sorted nitrogen atoms.")

# Example
reorder_nitrogens_by_reference('ZrN.xyz', 'ZrN_sorted.xyz', metal_symbol='Zr')


ImportError: cannot import name 'get_fractional_coordinates' from 'ase.geometry' (c:\Users\ameer\miniforge3\envs\ase\Lib\site-packages\ase\geometry\__init__.py)

In [15]:
view(read('TiN.xyz'))  # Visualize the reordered crystal structure for TiN
view(read('VN.xyz'))  # Visualize the reordered crystal structure for VN
view(read('ScN.xyz'))  # Visualize the reordered crystal structure for ScN   
view(read('NbN.xyz'))  # Visualize the reordered crystal structure for NbN
view(read('ZrN.xyz'))  # Visualize the reordered crystal structure for ZrN

<Popen: returncode: None args: ['c:\\Users\\ameer\\miniforge3\\envs\\ase\\py...>

In [20]:
from ase.io import read, write
from ase.build import surface
from ase.build.tools import sort
from ase.constraints import FixAtoms

bulk = read('ZrN.xyz')
view(bulk)
# Create (100) surface, 2x2 size, 4 atomic layers
slab = surface(bulk, (1, 0, 0), layers=2, vacuum=5.0, periodic=True)
view(slab)  # This will open a window to visualize the slab structure
slab *= (2, 2, 1)  # Expand in x and y
view(slab)
# Optional: sort atoms by z-position (not strictly necessary but helps visualization)
sort(slab)

# Optional: Fix bottom 2 layers
z_positions = [atom.z for atom in slab]
cutoff = sorted(z_positions)[len(z_positions)//4 * 2]  # bottom half
constraint = FixAtoms(indices=[atom.index for atom in slab if atom.z <= cutoff])
slab.set_constraint(constraint)

# Save the slab structure
write('ZrN_100_slab.xyz', slab)
# Visualize the slab structure
